In [1]:
from strsimpy.normalized_levenshtein import NormalizedLevenshtein

normalized_levenshtein = NormalizedLevenshtein()
print(normalized_levenshtein.distance('My string', 'My $string'))
print(normalized_levenshtein.distance('My string', 'My $string'))
print(normalized_levenshtein.distance('My string', 'My $string'))

print(normalized_levenshtein.similarity('My string', 'My $string'))
print(normalized_levenshtein.similarity('My string', 'My $string'))
print(normalized_levenshtein.similarity('My string', 'My $string'))
# get original transcript

transcript_id = '10_0991_331330'
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import WhitespaceTokenizer

def read_transcript_from_id(transcript_id):

    path_to_data_folder = '/archive/shared/sim_center/shared/ameer/'
    # lookinto this dictionary to find the path
    # can also manually create the path and it would be faster but not by much
    merged_lookup = pd.read_csv(path_to_data_folder + 'grade_lookupv5.csv')
    transcript = ''

    path = merged_lookup[merged_lookup.id == transcript_id].path.iloc[0]
    with open(path, 'r') as file:
        lines = file.readlines()
    for line in lines:
        if line != '\n':
            tok_line = line.split(' ')
            for i in range(len(tok_line)):
                transcript += ' ' + tok_line[i]


    return transcript
    
transcript = read_transcript_from_id(transcript_id)
transcript
diarized_transcript = '/archive/shared/sim_center/shared/annie/GPT4 3-chunk/'
id = '10_0991_331330'
def reconstruct_transcript(path, id): 
    transcript = ''
    path = path + id + '.txt'
    with open(path, 'r') as file:
        lines = file.readlines()
    for line in lines[1:]:
        if line != '\n':
            tok_line = line.split(' ')
            for i in range(1,len(tok_line)):
                transcript += ' ' + tok_line[i]
    
    resid_lines = transcript.split('\n')
    transcript = ''
    for line in resid_lines:
        transcript += line

    return transcript

reconstructed = reconstruct_transcript(diarized_transcript, id)
reconstructed
normalized_levenshtein.distance(transcript, reconstructed)
normalized_levenshtein.similarity(transcript, reconstructed)
normalized_levenshtein.distance(transcript, transcript)
#Run for all IDs
# IDs for the test/sample transcripts

ids = ['01_0542_298135',
'02_0036_174595',
'03_0028_174553',
'04_0043_174686',
'05_0033_174804',
'06_0079_175106',
'07_0068_174641',
'08_0029_174576',
'09_0029_174582',
'10_0991_331330']

gpt35_path = '/archive/shared/sim_center/shared/annie/GPT3.5 complete/'
mixtral_path = '/archive/shared/sim_center/shared/annie/mixtral/'
gpt4_path = '/archive/shared/sim_center/shared/annie/GPT4 complete (json split)/'
gpt4_3chunk = '/archive/shared/sim_center/shared/annie/GPT4 3-chunk/'
gpt4_6chunk = '/archive/shared/sim_center/shared/annie/GPT4 6-chunk/'
gpt4_9chunk = '/archive/shared/sim_center/shared/annie/GPT4 9-chunk/'

gpt35_init = []
mixtral_init = []
gpt4_init = []
gpt4_3chunk = []
gpt4_6chunk = []
gpt4_9chunk = []
def eval_transcript(ids, path):
    scores = {}
    for id in ids:
        print(id)
        transcript = read_transcript_from_id(id)
        reconstructed_transcript = reconstruct_transcript(path, id)
        distance = normalized_levenshtein.distance(transcript, reconstructed_transcript)
        similarity = normalized_levenshtein.similarity(transcript, reconstructed_transcript)
        scores[id] = [distance, similarity]
    return scores
    
gpt4_init = eval_transcript(ids, gpt4_path)
gpt4_init
gpt4_df = pd.DataFrame(gpt4_init)
gpt4_df.to_csv("gpt4_init.csv")
mixtral_init = eval_transcript(ids, mixtral_path)
pd.DataFrame(mixtral_init).to_csv("mixtral_init.csv")
gpt35_init = eval_transcript(ids, gpt35_path)
pd.DataFrame(gpt35_init).to_csv("gpt35_init.csv")
gpt4_dists = []
gpt35_dists = []
mixtral_dists = []
gpt4_sim = []
gpt35_sim = []
mixtral_sim = []

for id in ids:
    gpt4_sim.append(gpt4_init[id][1])
    gpt4_dists.append(gpt4_init[id][0])
    gpt35_sim.append(gpt35_init[id][1])
    gpt35_dists.append(gpt35_init[id][0])
    mixtral_sim.append(mixtral_init[id][1])
    mixtral_dists.append(mixtral_init[id][0])

gpt4_sim
columns = ['COSCE ID', 'gpt4 sim', 'gpt4 dist', 'gpt35 sim', 'gpt35 dist', 'mixtral sim', 'mixtral dist']
all_df = pd.DataFrame(zip(ids, gpt4_sim, gpt4_dists, gpt35_sim, gpt35_dists, mixtral_sim, mixtral_dists), columns=columns)
all_df
all_df.to_csv("levenshtein_gpt4_35_mixtral.csv")
import matplotlib.pyplot as plt
import numpy as np
avg_dists = [np.mean(gpt4_dists), np.mean(gpt35_dists), np.mean(mixtral_dists)]
avg_sim = [np.mean(gpt4_sim), np.mean(gpt35_sim), np.mean(mixtral_sim)]
models = ['gpt4', 'gpt35', 'mixtral']
plt.bar(models, avg_dists)
plt.ylim(0, 1)
plt.title("normalized levenshtein distance")
plt.show()
plt.bar(models, avg_sim)
plt.ylim(0, 1)
plt.title("normalized levenshtein similarity")
plt.show()

0.1
0.1
0.1
0.9
0.9
0.9


KeyboardInterrupt: 